In [2]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

In [128]:
gg = pd.read_csv('films_globe.csv', index_col=0)
res = []
for year in gg.index.unique():
    temp = gg.loc[year]
    if type(temp) == pd.core.series.Series:
        res += [1]
    else: 
        res += [1] + [0 for i in range(len(temp) -1 )]
res = np.array(res)
gg['Globe winner'] = res

In [129]:
gg.head(10)

,Film,Director,Producers,Globe winner
Year,,,,
1951,A Place in the Sun,George Stevens,George Stevens,1
1951,Bright Victory,Mark Robson,Robert Buckner,0
1951,Detective Story,William Wyler,William Wyler,0
1951,Quo Vadis,Mervyn LeRoy,Sam Zimbalist,0
1951,A Streetcar Named Desire,Elia Kazan,Charles K. Feldman,0
1952,The Greatest Show on Earth,Cecil B. DeMille,Cecil B. DeMille,1
1952,"Come Back, Little Sheba",Daniel Mann,Hal B. Wallis,0
1952,The Happy Time,Richard Fleischer,Earl Fenton,0
1952,High Noon,Fred Zinnemann,Stanley Kramer,0


In [132]:
bf = pd.read_csv("films_bafta.csv", index_col=0)
res = []
for year in bf.index.unique():
    temp = bf.loc[year]
    if type(temp) == pd.core.series.Series:
        res += [1]
    else: 
        res += [1] + [0 for i in range(len(temp) -1 )]
res = np.array(res)
bf['BAFTA winner'] = res

In [133]:
bf.head()

,name,BAFTA winner
year,,
1960,The Apartment,1
1960,The 400 Blows,0
1960,The Angry Silence,0
1960,The Adventure,0
1960,La Dolce Vita,0


In [134]:
import urllib
import re
from bs4 import BeautifulSoup as bs

url = 'https://en.wikipedia.org/wiki/Critics%27_Choice_Movie_Award_for_Best_Picture'
html = urllib.request.urlopen(url)
soup = bs(html, 'html.parser')

In [137]:
res = soup.findAll('ul')
for item in res:
    if '1995' in item.find('li').contents or '2010' in item.find('li').contents or '2010' in item.find('li').contents:
        data = item

AttributeError: 'NoneType' object has no attribute 'contents'

In [139]:
"""Create CSVs from all tables on a Wikipedia article."""

import csv
import os
import platform

from bs4 import BeautifulSoup
import requests

def scrape(url="https://en.wikipedia.org/wiki/Screen_Actors_Guild_Award_for_Outstanding_Performance_by_a_Female_Actor_in_a_Leading_Role", output_name='actress'):
    """Create CSVs from all tables in a Wikipedia article.

    ARGS:
        url (str): The full URL of the Wikipedia article to scrape tables from.
        output_name (str): The base file name (without filepath) to write to.
    """

    # Read tables from Wikipedia article into list of HTML strings
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    # table_classes = {"class": ["sortable", "plainrowheaders"]}
    table_classes = {'class': 'wikitable'}
    wikitables = soup.findAll("table", table_classes)[0:3]
    print (len(wikitables))

    # Create folder for output if it doesn't exist
    try:
        os.mkdir(output_name)
    except Exception:  # Generic OS Error
        pass

    for index, table in enumerate(wikitables):
        # Make a unique file name for each CSV
        if index == 0:
            filename = output_name
        else:
            filename = output_name + '_' + str(index)

        filepath = os.path.join(output_name, filename) + '.csv'

        with open(filepath, mode='w', newline='', encoding='utf-8') as output:
            # Deal with Windows inserting an extra '\r' in line terminators
            if platform.system() == 'Windows':
                kwargs = {'lineterminator': '\n'}

            csv_writer = csv.writer(output, quoting=csv.QUOTE_ALL)
            write_html_table_to_csv(table, csv_writer)


def write_html_table_to_csv(table, writer):
    """Write HTML table from Wikipedia to a CSV file.

    ARGS:
        table (bs4.Tag): The bs4 Tag object being analyzed.
        writer (csv.writer): The csv Writer object creating the output.
    """

    # Hold elements that span multiple rows in a list of
    # dictionaries that track 'rows_left' and 'value'
    saved_rowspans = []
    for row in table.findAll("tr"):
        cells = row.findAll(["th", "td"])

        # If the first row, use it to define width of table
        if len(saved_rowspans) == 0:
            saved_rowspans = [None for _ in cells]
        # Insert values from cells that span into this row
        elif len(cells) != len(saved_rowspans):
            for index, rowspan_data in enumerate(saved_rowspans):
                if rowspan_data is not None:
                    # Insert the data from previous row; decrement rows left
                    value = rowspan_data['value']
                    cells.insert(index, value)

                    if saved_rowspans[index]['rows_left'] == 1:
                        saved_rowspans[index] = None
                    else:
                        saved_rowspans[index]['rows_left'] -= 1

        # If an element with rowspan, save it for future cells
        for index, cell in enumerate(cells):
            if cell.has_attr("rowspan"):
                rowspan_data = {
                    'rows_left': int(cell["rowspan"]),
                    'value': cell,
                }
                saved_rowspans[index] = rowspan_data

        if cells:
            # Clean the data of references and unusual whitespace
            cleaned = clean_data(cells)

            # Fill the row with empty columns if some are missing
            # (Some HTML tables leave final empty cells without a <td> tag)
            columns_missing = len(saved_rowspans) - len(cleaned)
            if columns_missing:
                cleaned += [None] * columns_missing

            writer.writerow(cleaned)


def clean_data(row):
    """Clean table row list from Wikipedia into a string for CSV.

    ARGS:
        row (bs4.ResultSet): The bs4 result set being cleaned for output.

    RETURNS:
        cleaned_cells (list[str]): List of cleaned text items in this row.
    """

    cleaned_cells = []

    for cell in row:
        # Strip references from the cell
        references = cell.findAll("sup", {"class": "reference"})
        if references:
            for ref in references:
                ref.extract()

        # Strip sortkeys from the cell
        sortkeys = cell.findAll("span", {"class": "sortkey"})
        if sortkeys:
            for ref in sortkeys:
                ref.extract()

        # Strip footnotes from text and join into a single string
        text_items = cell.findAll(text=True)
        no_footnotes = [text for text in text_items if text[0] != '[']

        cleaned = (
            ''.join(no_footnotes)  # Combine elements into single string
            .replace('\xa0', ' ')  # Replace non-breaking spaces
            .replace('\n', ' ')  # Replace newlines
            .strip()
        )

        cleaned_cells += [cleaned]

    return cleaned_cells


if __name__ == '__main__':
    print('Running Main!!!')

Running Main!!!


In [140]:
url = 'https://en.wikipedia.org/wiki/Independent_Spirit_Award_for_Best_Film'
scrape(url, output_name='film')


2


IndexError: list assignment index out of range